In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold
import re

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import tensorflow as tf
import keras

Using Theano backend.


In [3]:
data_path = '/home/raku/kaggleData/2sigma/lr4/'

train_df=pd.read_json(data_path+'lr4-n-train.json')
test_df=pd.read_json(data_path+'lr4-n-test.json')



In [4]:
pickl_file = '/home/raku/kaggleData/2sigma/loglr/'+'loglrfeatures.pickle'
fileObject = open(pickl_file,'r') 
features=pickle.load(fileObject)   
fileObject.close()
print len(features)

223


In [5]:
numericals = [u'bath_per_bed',u'bathrooms',u'bedrooms',u'building0',u'cluster_id_10_d',u'cluster_id_30_d',u'dayofyear',
 u'latitude',u'listing_id',u'longitude',u'm14perf',u'm14perf_f',u'm30perf',u'm30perf_f',u'm3perf',u'm3perf_f',
 u'm7perf',u'm7perf_f',u'm_c_distance',u'm_m_distance',u'manager_id_nrank',u'manager_id_perf',u'mlat',
 u'mlon',u'num_description_words',u'num_features', u'num_photos',
 u'price',u'price_per_bath',u'price_per_bed',u'price_per_room',]
len(numericals)

31

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras import optimizers
from keras.callbacks import EarlyStopping

In [26]:
def nn_model(features,num_classes=3,lr=0.1):
    model = Sequential()
    model.add(Dense(64,  
                    activation='softplus',
                    input_shape = (len(features),),
                                  kernel_initializer='he_normal',
                                  kernel_regularizer=l2(0.000025)
                                  ))
    model.add(Dropout(0.2))
    
    model.add(Dense(16,
                    activation='softplus', 
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(0.000025)
                    ))
    model.add(Dropout(0.1))

    model.add(Dense(units=num_classes, 
                    activation='softmax', 
                    kernel_initializer='he_normal',
                    ))
    opt = optimizers.Adadelta(lr=1)
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=opt,
                  metrics=['accuracy']
                  )
    return(model)

In [11]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)



In [28]:
for batch_size in [64,128]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
    cv_scores=[]
    for dev_index, val_index in KF:
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
    
        early_stopping = EarlyStopping(monitor='val_loss', patience=20)

        seed = 0
        np.random.seed(seed)
        model = nn_model(features,lr=0.1)
        model.fit(dev_X, dev_y, epochs = 1000, batch_size=batch_size, verbose = 2, 
          validation_data=[val_X, val_y], callbacks=[early_stopping])

        preds =  model.predict_proba(val_X)
        
        cv_scores.append(log_loss(val_y, preds))
        
        print(cv_scores)
    
    print np.mean(cv_scores)


Train on 39481 samples, validate on 9871 samples
Epoch 1/1000
8s - loss: 0.7141 - acc: 0.7015 - val_loss: 0.6133 - val_acc: 0.7320
Epoch 2/1000
7s - loss: 0.6272 - acc: 0.7243 - val_loss: 0.6017 - val_acc: 0.7336
Epoch 3/1000
7s - loss: 0.6129 - acc: 0.7295 - val_loss: 0.5978 - val_acc: 0.7345
Epoch 4/1000
7s - loss: 0.6057 - acc: 0.7335 - val_loss: 0.5953 - val_acc: 0.7352
Epoch 5/1000
9s - loss: 0.6005 - acc: 0.7357 - val_loss: 0.5932 - val_acc: 0.7354
Epoch 6/1000
8s - loss: 0.5961 - acc: 0.7356 - val_loss: 0.5921 - val_acc: 0.7369
Epoch 7/1000
7s - loss: 0.5941 - acc: 0.7372 - val_loss: 0.5912 - val_acc: 0.7399
Epoch 8/1000
7s - loss: 0.5946 - acc: 0.7357 - val_loss: 0.5899 - val_acc: 0.7407
Epoch 9/1000
8s - loss: 0.5910 - acc: 0.7373 - val_loss: 0.5895 - val_acc: 0.7394
Epoch 10/1000
8s - loss: 0.5875 - acc: 0.7381 - val_loss: 0.5894 - val_acc: 0.7408
Epoch 11/1000
8s - loss: 0.5861 - acc: 0.7418 - val_loss: 0.5881 - val_acc: 0.7394
Epoch 12/1000
9s - loss: 0.5874 - acc: 0.7384 -

KeyboardInterrupt: 

In [ ]:
for batch_size in [256,512]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
    cv_scores=[]
    for dev_index, val_index in KF:
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
    
        early_stopping = EarlyStopping(monitor='val_loss', patience=20)

        seed = 0
        np.random.seed(seed)
        model = nn_model(features,lr=0.1)
        model.fit(dev_X, dev_y, epochs = 1000, batch_size=batch_size, verbose = 2, 
          validation_data=[val_X, val_y], callbacks=[early_stopping])

        preds =  model.predict_proba(val_X)
        
        cv_scores.append(log_loss(val_y, preds))
        
        print(cv_scores)
    
    print np.mean(cv_scores)